In [ ]:
# default_exp resnet

# resnet

> Resnet family models.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# export
import torch.nn as nn
import torch
from collections import OrderedDict

In [ ]:
# export
from model_constructor.constructor import *
from model_constructor.layers import *

# Bottleneck

In [ ]:
# export
class Bottleneck(nn.Module):
    '''Bottlneck block for resnet models'''
    def __init__(self, ni, nh, stride=1, bn_1st=False, zero_bn=False,
#                  groups=1, base_width=64, dilation=1, norm_layer=None
                expansion=4, **kwargs):
        super().__init__()
        self.downsample = not ni==nh or stride==2
        ni = ni*expansion
        nf = nh*expansion
        self.conv = nn.Sequential(OrderedDict([
            ('conv_0', ConvLayer(ni, nh, ks=1,          bn_1st=bn_1st, **kwargs)),
            ('conv_1', ConvLayer(nh, nh, stride=stride, bn_1st=bn_1st, **kwargs)),
            ('conv_2', ConvLayer(nh, nf, ks=1, zero_bn=zero_bn, bn_1st=bn_1st, **kwargs))]))
        if self.downsample:
            self.downsample = ConvLayer(ni, nf, ks=1, stride=stride, act=False, **kwargs)
        self.merge = Noop()
        self.act_conn = act_fn

    def forward(self, x):
        identity = x
        out = self.conv(x)
        if self.downsample:
            identity = self.downsample(x)
        return self.act_conn(self.merge(out + identity))

In [ ]:
b_block = Bottleneck(16,64)
b_block

Bottleneck(
  (conv): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (downsample): ConvLayer(
    (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
b_block = Bottleneck(64,64)
b_block

Bottleneck(
  (conv): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
body = Body(Bottleneck, expansion=4)

In [ ]:
body

Body(
  (layer_0): Sequential(
    (block_0): Bottleneck(
      (conv): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (downsample): ConvLayer(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=Tru

In [ ]:
xb = torch.randn(64, 64, 32, 32)
y = body(xb)
y.shape

torch.Size([64, 2048, 4, 4])

# Resnet models with BasicBlock/

In [ ]:
# export
def resnet18(**kwargs):
    """Constructs a ResNet-18 model. """
    return Net(block=BasicBlock, blocks=[2, 2, 2, 2], **kwargs)
def resnet34(**kwargs):
    """Constructs a ResNet-18 model. """
    return Net(block=BasicBlock, blocks=[3, 4, 6, 3], **kwargs)

In [ ]:
resnet18()

Net(
  (stem): Stem(
    sizes: [3, 64]
    (conv0): ConvLayer(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): Sequential(
      (block_0): BasicBlock(
        (conv): Sequential(
          (conv_0): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act_fn): ReLU(inplace=True)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (conv_1): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act_fn): ReLU(inplace=True)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         

In [ ]:
resnet34()

Net(
  (stem): Stem(
    sizes: [3, 64]
    (conv0): ConvLayer(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): Sequential(
      (block_0): BasicBlock(
        (conv): Sequential(
          (conv_0): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act_fn): ReLU(inplace=True)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (conv_1): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act_fn): ReLU(inplace=True)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         

# Resnet50

In [ ]:
# export
def resnet50(**kwargs):
    """Constructs a ResNet-18 model. """
    return Net(block=Bottleneck, expansion=4, blocks=[3, 4, 6, 3], **kwargs)

In [ ]:
model = resnet50()

In [ ]:
model.body

Body(
  (layer_0): Sequential(
    (block_0): Bottleneck(
      (conv): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (downsample): ConvLayer(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=Tru

In [ ]:
xb = torch.randn(64, 3, 128, 128)
y = model(xb)
y.shape

torch.Size([64, 1000])

# end. resnet.
model_constructor
by ayasyrev

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_constructor.ipynb.
Converted 01_layers.ipynb.
Converted 02_resnet.ipynb.
Converted index.ipynb.
